# Light up the world, precious nerds!

Why stumble around in the dark when you need to visit the bathroom at night?
Make the Pi turn on lights upon detecting motion in the dark.
The GPIO pins interface with sensors to give the Pi eyes and ears.
Let us see how as we work through the schematics.


## 1. What is GPIO?

A maker project is a system with components that exchange messages with the Pi. Sensors convey messages about external conditions like temperature, light and pressure. An actuator, such as motor, receives a message to energize. The GPIO pins provide the interface for exchanging messages. 

In the old days, a taxi-meter had a flag to indicate whether a taxi was vacant or not. Once the taxi had a fare, the operator manually rotated the flag to indicate the taxi was not available any more. A GPIO pin has a similar function as the taxi-meter flag. It conveys a simple dichotomous message - high or low. ![Flag](images/taxi_meter_flag.jpg)

When a sensor conveys information to the Pi, it puts a "high" or "low" message on a GPIO pin. Thus a temperature sensor may indicate high or low temperature. Or a moisture sensor may indicate dry or wet soil. To recieve information, a GPIO pin must be in listening mode, i.e. "input".

Similarly, the Pi can energize an actuator by putting a "high" or "low" message on a GPIO pin. GPIO "high" and "low" signals tell hardware to switch on and off. To send information, a GPIO pin must be in transmitting mode, i.e. "output". 

*A GPIO message is constructed from electrical voltage. Communication requires that the sender and receiver speak the same language. Let us see how electrical hardware and the Pi communicate in a common language using voltage, starting with the simplest case.*

### GPIO Output: Blink an LED

A Pi speaks "high" or "low" in electrical voltage. A voltage of 3.3 V is "high". A voltage of 0 V is "low". The 3.3 V on the pin is sufficient to power a small LED light. Wire up the circuit like so. ![BlinkLED](images/ozWiz_dancingLights_e_blinkLED.png)

The schematic shows a Raspberry Pi with a cartoon of the GPIO pins as they appear on the board. The pins are numbered 1 through 40 in the order of appearance. Some pins simple serve to provide an output voltage (3.3 V or 5 V) and a reference 0 V (Ground). A few GPIO pins haver additional specialized uses in communicating complex messages based on timing of "high" and "low" pulses. Pins also have a GPIO ID number that the Maker can use when refering to pin numbers in code, being careful to specify upfront which numbering scheme is in use.

Future schematics will not show the Raspberry Pi. The following simplification is sufficient. ![simpleSchematic](images/ozWiz_dancingLights_e_blinkLED_simple.png)

Note that:
- The pin no. is the GPIO ID. 
- By convention, red wire is the "live" feed and black wire is the return path.
- Other circuitry components are depicted with standard symbols where available or representative images.

The **Pi** puts out is a logic level "high" at GPIO#24. The **LED** receives the "high" from Pi at the anode (long leg) and provides an electrical path to the ground (GND) at the cathode (short leg).

In [1]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/rMdXbVPZ60w" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

*A voltage of 3.3 V is enough to power on an LED. However, the power requirements of devices are often beyond what the Pi itself can supply. We need to add additional "glue" circuity to operate devices with independent power supply. In the next section we shall see a few different ways of doing that.* 

## 2. Relay GPIO Signals to High-Voltage Devices

So far, we have operated an LED from the Pi using a GPIO pin. The logical "high" at 3.3 V was sufficient to power the LED which requires very little power to run. In practical applications, the GPIO pin is rarely used to supply a device directly. A more general approach is to supply a device's power requirement independently of the Pi and use GPIO to operate an on-off switch in a device's voltage feed to control it. This approach is versatile and opens up the possibility of controlling devices with power requirement beyond that which the Pi can supply, including home appliances (like a gas furnace or a washing machine) that draw power from the mains.

A relay represents realization of this principle. The simplest relays are of electro-mechanical design where an electromagnet closes electrical contacts against opposing spring action. Relays are a popular choice among makers for home automation projects and are available as a "hardware attached on top" (HAT) for the Pi. 

Electromechanical Relay | Pi Hat with Relays
- | -
![Relay Module](images/relayModule.jpg) | ![piHat](images/raspberryPi_relayHat.jpg)

With the addition of a relay, the circuit to operate our LED looks like so. Note the modification to operate a lamp supplied from the mains. Substitute the lamp for any household device supplied by the mains to explore the versatility of this approach. 

Relay to LED | Relay to Lamp
- | - 
![Relay to LED](images/ozWiz_dancingLights_e_relayLED.png) | ![Relay to Lamp](images/ozWiz_dancingLights_e_relayMains.png)
*Operate an LED using GPIO via relay.*  | *Modify the LED circuit to operate a lamp that draws power from the mains supply.*

What is going on here? Let's examine what goes in and what comes out for each device.

The **Pi** outputs "high" at GPIO#24. This part remains the same whether operating LED or Lamp. When the **relay** sees a "high" from the Pi at pin S (signal), The relay shorts the two pins marked "normally open" (NO) at the output. The **LED** or **Lamp** has the "live" feed "spliced in" the NO terminals so that upon shorting, the electrical path is completed and the device comes on. 

**Note:**
1. The relay has similar power requirements as the Pi and draws from the Pi's source via pins VCC (positive terminal) and GND (negative terminal).
2. The return path for the lamp is the neutral of the mains and that for the LED is the ground.
3. The relay is "spliced in" the cable used to plug in the lamp to the electrical outlet.

*Relays make it possible to use a Pi to control electrical hardware with power requirements beyond the ability of the Pi to supply. However, the mechanical nature of operation produces wear and tear and contacts wear out from sparking. What if there was an alternative without moving parts? Next, we'll look at how MOSFETs are just the thing!*

### Use a MOSFET as switching element

A relay is an electrically-operated switch. Its low power-requirement lends it versatility. However, the moving parts within a relay pose an Achilles heel. This is where MOSFETs come in.

A metal-oxide-semiconductor field-effect-transistor (MOSFET) is a voltage-operated electrical switch. (More precisely, a MOSFET is a voltage-controlled resistor.) The business end of the MOSFET has three terminals: Gate, Source and Drain. They are shown labeled in the figures, in the schematic representation as well as a popular "through hole" package. 

MOSFET Schematic | MOSFET Package
- | -
![MOSFET symbol](images/OzWiz_MOSFETschematic.png) | ![MOSFET package](images/MOSFET_package.jpg)
Use the voltage difference between gate (G) and source (S) to regulate the resistance between drain (D) and source (S),thus using the MOSFET as an electrically-controlled switch. | Raise gate voltage (left-most pin) above the source voltage (right-most pin) by the threshold or more to short the drain (middle pin) and source. 

When the voltage at the gate exceeds the voltage at source by a threshold, the source and drain terminals are shorted. With the source connected to ground, that means the voltage applied at gate should exceed the threshold. When the gate voltage is below threshold, the source and drain are open. Use the MOSFET to interface an electrical load with GPIO like so.

MOSFET Usage Pattern |  MOSFET Pin-Out for Through-Hole Package
- | -
![MOSFET](images/OzWiz_MOSFETcircuit.png) | ![MOSFET circuit](images/OzWiz_MOSFETcircuit_package.png)

The **Pi** outputs a "high" (or "low") at the GPIO pin. The **MOSFET** sees the "high" (or "low") at the gate and shorts (or opens) the source and drain. The **load** is placed on the MOSFET's drain side with the power supply. The source provides the path to ground. When the drain and source are shorted, the electrical path is compelted and the load comes on. 

It is worth nothing that the configuration shown is typical of a type "n-channel" (as opposed to p-channel) MOSFET. The load is placed on the other side in case of a p-channel MOSFET.

**Note:** Choose a MOSFET to the correct specification. 
1. The Pi's microcontroller uses 3.3 V logic (i.e. 3.3 V is "high"). Other microcontrollers, such as the popular Arduino, use a 5 V logic. Be sure to use a MOSFET compatible with 3.3 V logic. 
2. We have shown an n-channel MOSFET with load at the drain and sink at the source. This is reversed for a p-channel MOSFET which has the load at the source and sink at the drain.
3. MOSFETs come packaged in through-hole and surface-mount configurations. The through-hole package shown has three leads as shown and the pin-out for the surface-mount configuration will vary. 

*So far, we have an approach to interface Pi with electrical hardware powered by battery or mains to automate on-off switching. This approach is versatile and will work with most household appliances. But how to sense events to switch devices on-off in response to? For that, we need to learn about sensors and how to use them with the Pi.*

## 3. Detect Motion in Darkness

A sensor needs to "speak" the logic level that the Pi's GPIO can understand and process. The sensor may come as a package with the electronic circuitry for use with the Pi. In that case, the maker's job is simply to make the right connections between the sensor and GPIO.

A maker must be prepared to build the "glue" circuity if needed. We will see how when working with a light-dependent resistor to sense lighting condition. We will then learn about the passive infra-red sensor which comes as a ready-to-use package for detecting motion. 

### Voltage-Divider Circuit

The endless variety and varying complexity of electrical circuits can seem daunting to the initiate. Over time, a maker learns to identify and catalog patterns that are used as building bricks. One such pattern is the voltage-divider circuit.

The voltage-divider is used to tap off a voltage at the desired fraction of the source, like so. ![Voltage Divider](images/ozWiz_voltageDivider.png)

The voltage at tap (Vo) is in proportion to the source (Vi) as the resistor (R2) is in proportion to the total resistance (R1+R2). In other words, Vout is a fraction of Vin as given by R2/(R1+R2). 

In engineering, it is often helpful to examine a phenomenon by taking it to extremes. 
- Let's take R2 to an extreme and say it has infinite resistance. So R1 is insignificant compared to R2 and so, (R1+R2) is as good as R2. Therefore, the entire voltage-drop occurs over R2 and Vo equals Vi. 
- Let's take R2 to the other extreme and say it has zero resistance. So Vo is connected to ground and the entire voltage-drop occurs over R1. So, Vo equals zero.

*How to put the voltage-divider circuit to practice use? The sensing element of a sensor often has an electrical property that changes in response to the sensory input. Say this property is electrical resistance. Then, using the voltage-divider circuit, it resistance can be converted to a voltage appropriate for a GPIO pin. Let us see how with a light-dependent resistor.*

### Light-Dependent Resistor (LDR)

A light-dependent resistor or photoresistor is a resistor that changes resistance in response to light. It has low resistance in light and high resistance in dark. The range (minimum and maximum) can vary by orders of magnitude, for example, from a few 100 Ohms in light to a few 1000k Ohms in dark. To sense light conditions, use a photoresistor in series with a large resistor in voltage divider configuration like so.

Photoresistor in Voltage-Divider | Photoresistor Pin-Out 
- | -
![Photoresistor](images/ozWiz_voltageDivider_photoresistor.png) | ![LDR](images/ozWiz_voltageDivider_ldr.png)

In the dark, the photoresistor behaves approximately like an open circuit, so the voltage divider's output voltage (Vo) is the same as the input voltage (Vi). In light, the photoresistor behaves approximately like a short-circuit, so the voltage divider's output voltage is ground voltage, i.e zero. Thus, use of photoresistor in voltage divider configuration enables representing light conditions as voltage levels for GPIO. 

*A voltage divider came in handy with a photoresistor's mode of operation. But in general, a sensor may use any electrical property of the sensing element to respond to stimulus. We need "glue" circuitry so the sensor outputs recognizable voltage levels. For this reason, sensors are often packaged with the circuitry for compatibility with popular protocols as we shall see next with the PIR sensor.*

### Passive Infra-Red Sensor (PIR)

The passive infra-red sensor (PIR) sensor is a type of light sensor tricked into detecting motion. Think about a solar cell that converts the sun's light into electrical energy. The infra-red sensor is like a solar cell but tuned to respond to infra-red light, i.e. light adjacent to color red in the VIBGYOR spectrum. This is light emitted by warm bodies of people and animals.

The shadow of a passing cloud reduces the electrical energy generated by a solar cell. Two identical solar cells slotted side-side and compared detects motion. This is the principle by which an infra-red sensor detects motion of people or animals. 

The sensor package has a lens so the sensing element can "see" farther out. Otherwise, only movement directly in front of the sensing element would register. 

PIR Package | PIR Pinout
- | - 
![PIR Package](images/ozWiz_PIRsensor.jpg) | ![PIR Pinout](images/ozWiz_PIRsensor.jpg)


The figure illustrates the use of the sensor. The circuit requires a 5 V supply to run. The output voltage is compatible with the Pi's 3.3 V logic levels. 

*We now have all the pieces we need to pull together into electrical schematics for our project. Let's proceed to do that next.*

## 4. Build and combine sub-assemblies

### Part A

*Circuit Diagram:*
![Party lights with relay HAT](images/ozWiz_partyLights_relayHAT.png)

*How it works:*
Affix the Pi HAT to the Pi, mating the GPIO socket header on the HAT to the GPIO header on the Pi. This results in the three relays labeled 03, 02 and 01 on the HAT being wired to GPIO pins 21, 20 and 26. (Note the order of labeling.) The mains have live and neutral lines. Connect the three bulbs in parallel. For each light bulb, splice the live feed into the normally open (NO) terminals of a relay and provide a return path to the neutral.  

### Part B


#### Sub-assembly - Light Sensor

*Circuit diagram:*
![Light Sensor Sub-Assembly](images/ozWiz_LDRmodule.png)

*How it works:*
Make a voltage-divider with the light-dependent resistor (LDR) in series with a 10k Ohms potentiometer. Set the input voltage at 5 V and potentiometer at 5k Ohms. Drive a n-channel MOSFET from the tap of this voltage-divider. The poteniometer is so that the sensor's operation can be fine-tuned. Make another voltage-divider with two 5k Ohms resistors in series. Set the input voltage at 5 V and attach the MOSFET's drain at the tap, the source attached to ground. *Be sure to tie all grounds together.* The voltage-divider's tap is the sensor's output. 

Assume that the drain and source are either shorted or open. When open, the two resistors split the input voltage into two so the output voltage is half the input voltage i.e. 2.5 V. When shorted, the tap is pulled to ground and the output voltage is 0 V. This is shown in the table accompanying the circuit diagram.

#### Sub-assembly - LED Lighting

*Circuit diagram:*
![RGB LED Strip](images/ozWiz_RGBcontrol.png)

*How it works:*
Repeat the application shown in 2. above three times for three LEDs, each of a different color. Each LED is controlled independently by a GPIO pin via MOSFET. An RGB LED strip will present the same configuration as three separate LEDs, with a common anode and an independent cathode each for grounding. 

#### Main Assembly

*Cicruit diagram:*
Integrate the PIR sensor, the light sensor sub-assembly and the LED lighting sub-assmebly with the Pi using a Pi Cobbler breakout for GPIO. Use the GPIO pin 

*How it works:*



## References:

In [16]:
%%HTML
<h4>Get the inside scoop on an electromechanical relay.</h4>
<iframe width="560" height="315" src="https://www.youtube.com/embed/MJz2DRZ1w9E" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

In [11]:
%%HTML
<h4>Use a MOSFET as a voltage controlled resistor in automation.</h4>
<iframe width="560" height="315" src="https://www.youtube.com/embed/GrvvkYTW_0k" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

In [14]:
%%HTML
<h4>Divide and conquer electrical circuits with the voltage-divider.</h4>
<iframe width="560" height="315" src="https://www.youtube.com/embed/t_hPrz7rs34" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

In [15]:
%%HTML
<h4>Take the vibe up a notch with party lights using relays on a Pi HAT.</h4>
<iframe width="560" height="315" src="https://www.youtube.com/embed/bOGltcgiXiU" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>